In [ ]:
print(analysis_df.dtypes)

experiment_duration               object
user_id                            int64
enthusiast                         int64
age                                int64
gender                             int64
education                          int64
region                             int64
openness                         float64
conscientiousness                float64
Extroversion                     float64
agreeableness                    float64
neuroticism                      float64
Familiarity_SelectedSong           int64
PleasantSurprise_SelectedSong      int64
MatchInterests_SelectedSong        int64
selected_recommendation_id         int64
selected_position                  int64
explanation                       object
explanation type                  object
confidence                         int64
usefulness                         int64
persuasion                         int64
transparency                       int64
comprehensibility                  int64
fairness        

In [ ]:


def run_explanation_stats(df, dimension, correction_method='holm', verbose=True):

    traits = ['Agreeableness', 'Conscientiousness', 'Extroversion', 'Neuroticism', 'Openness', 'FeatureBased']
    columns = [f"{dimension}_{trait}" for trait in traits]

    # Check columns exist
    for col in columns:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in DataFrame.")


    friedman_data = [df[col].dropna() for col in columns]
    stat, p = friedmanchisquare(*friedman_data)

    if verbose:
        print(f"--- {dimension.upper()} ---")
        print(f"Friedman test statistic: {stat:.2f}")
        print(f"p-value: {p:.4f}")

    results = []
    raw_p_values = []
    comparisons = list(combinations(columns, 2))

    for col1, col2 in comparisons:
        data = df[[col1, col2]].dropna()
        if len(data) == 0:
            continue  # Skip empty pairs
        stat, p = wilcoxon(data[col1], data[col2])
        results.append({
            'Comparison': f"{col1} vs {col2}",
            'W-statistic': stat,
            'Raw p-value': p
        })
        raw_p_values.append(p)


    _, corrected_pvals, _, _ = multipletests(raw_p_values, method=correction_method)

    for i, result in enumerate(results):
        result[f'Corrected p-value ({correction_method.capitalize()})'] = corrected_pvals[i]
        result['Significant'] = corrected_pvals[i] < 0.05

    posthoc_df = pd.DataFrame(results).sort_values(by=f'Corrected p-value ({correction_method.capitalize()})')

    if verbose:
        print(posthoc_df)

    return posthoc_df


In [ ]:
run_explanation_stats(analysis_df, 'quality')



--- QUALITY ---
Friedman test statistic: 34.97
p-value: 0.0000
                                           Comparison  W-statistic  \
8   quality_Conscientiousness vs quality_FeatureBased       4367.5   
0   quality_Agreeableness vs quality_Conscientious...       3564.5   
6    quality_Conscientiousness vs quality_Neuroticism       4091.0   
5   quality_Conscientiousness vs quality_Extroversion       4109.0   
14           quality_Openness vs quality_FeatureBased       4300.0   
12            quality_Neuroticism vs quality_Openness       4872.0   
3           quality_Agreeableness vs quality_Openness       6222.0   
10           quality_Extroversion vs quality_Openness       5774.5   
7       quality_Conscientiousness vs quality_Openness       5703.5   
4       quality_Agreeableness vs quality_FeatureBased       6263.0   
1       quality_Agreeableness vs quality_Extroversion       4875.5   
2        quality_Agreeableness vs quality_Neuroticism       5283.5   
11       quality_Extroversi

,Comparison,W-statistic,Raw p-value,Corrected p-value (Holm),Significant
8,quality_Conscientiousness vs quality_FeatureBased,4367.5,0.000022,0.000332,True
0,quality_Agreeableness vs quality_Conscientious...,3564.5,0.000024,0.000337,True
6,quality_Conscientiousness vs quality_Neuroticism,4091.0,0.000068,0.000880,True
5,quality_Conscientiousness vs quality_Extroversion,4109.0,0.000114,0.001367,True
14,quality_Openness vs quality_FeatureBased,4300.0,0.006184,0.068029,False
12,quality_Neuroticism vs quality_Openness,4872.0,0.025682,0.256825,False
3,quality_Agreeableness vs quality_Openness,6222.0,0.032746,0.294715,False
10,quality_Extroversion vs quality_Openness,5774.5,0.059790,0.478324,False
7,quality_Conscientiousness vs quality_Openness,5703.5,0.081381,0.569669,False
4,quality_Agreeableness vs quality_FeatureBased,6263.0,0.742752,1.000000,False


In [ ]:
run_explanation_stats(analysis_df, 'taste')

--- Accuracy ---
Friedman test statistic: 40.51
p-value: 0.0000
                                        Comparison  W-statistic  Raw p-value  \
6     taste_Conscientiousness vs taste_Neuroticism       4574.5     0.000002   
8    taste_Conscientiousness vs taste_FeatureBased       5894.0     0.000014   
12             taste_Neuroticism vs taste_Openness       4867.5     0.000097   
14            taste_Openness vs taste_FeatureBased       6391.5     0.000294   
0   taste_Agreeableness vs taste_Conscientiousness       5051.5     0.000642   
3            taste_Agreeableness vs taste_Openness       6622.0     0.008088   
9          taste_Extroversion vs taste_Neuroticism       5643.0     0.020789   
5    taste_Conscientiousness vs taste_Extroversion       6803.5     0.030704   
11        taste_Extroversion vs taste_FeatureBased       6745.0     0.047675   
10            taste_Extroversion vs taste_Openness       6923.5     0.141846   
1        taste_Agreeableness vs taste_Extroversion      

,Comparison,W-statistic,Raw p-value,Corrected p-value (Holm),Significant
6,taste_Conscientiousness vs taste_Neuroticism,4574.5,0.000002,0.000036,True
8,taste_Conscientiousness vs taste_FeatureBased,5894.0,0.000014,0.000200,True
12,taste_Neuroticism vs taste_Openness,4867.5,0.000097,0.001265,True
14,taste_Openness vs taste_FeatureBased,6391.5,0.000294,0.003528,True
0,taste_Agreeableness vs taste_Conscientiousness,5051.5,0.000642,0.007063,True
3,taste_Agreeableness vs taste_Openness,6622.0,0.008088,0.080884,False
9,taste_Extroversion vs taste_Neuroticism,5643.0,0.020789,0.187103,False
5,taste_Conscientiousness vs taste_Extroversion,6803.5,0.030704,0.245636,False
11,taste_Extroversion vs taste_FeatureBased,6745.0,0.047675,0.333724,False
10,taste_Extroversion vs taste_Openness,6923.5,0.141846,0.851077,False


In [ ]:

run_explanation_stats(analysis_df, 'listen', correction_method='bonferroni')

--- Persuasiveness ---
Friedman test statistic: 45.05
p-value: 0.0000
                                          Comparison  W-statistic  \
6     listen_Conscientiousness vs listen_Neuroticism       3948.0   
12             listen_Neuroticism vs listen_Openness       4661.0   
8    listen_Conscientiousness vs listen_FeatureBased       6638.0   
0   listen_Agreeableness vs listen_Conscientiousness       5223.0   
14            listen_Openness vs listen_FeatureBased       7346.0   
5    listen_Conscientiousness vs listen_Extroversion       6785.5   
3            listen_Agreeableness vs listen_Openness       6662.5   
9          listen_Extroversion vs listen_Neuroticism       6262.5   
10            listen_Extroversion vs listen_Openness       7576.5   
2         listen_Agreeableness vs listen_Neuroticism       5603.5   
1        listen_Agreeableness vs listen_Extroversion       6736.5   
4        listen_Agreeableness vs listen_FeatureBased       6926.0   
7        listen_Conscientiousness

,Comparison,W-statistic,Raw p-value,Corrected p-value (Bonferroni),Significant
6,listen_Conscientiousness vs listen_Neuroticism,3948.0,1.787378e-08,2.681066e-07,True
12,listen_Neuroticism vs listen_Openness,4661.0,9.171600e-06,1.375740e-04,True
8,listen_Conscientiousness vs listen_FeatureBased,6638.0,4.397329e-05,6.595994e-04,True
0,listen_Agreeableness vs listen_Conscientiousness,5223.0,2.599380e-04,3.899070e-03,True
14,listen_Openness vs listen_FeatureBased,7346.0,1.075290e-03,1.612935e-02,True
5,listen_Conscientiousness vs listen_Extroversion,6785.5,2.147475e-03,3.221212e-02,True
3,listen_Agreeableness vs listen_Openness,6662.5,1.910985e-02,2.866477e-01,False
9,listen_Extroversion vs listen_Neuroticism,6262.5,3.296545e-02,4.944817e-01,False
10,listen_Extroversion vs listen_Openness,7576.5,4.013297e-02,6.019945e-01,False
2,listen_Agreeableness vs listen_Neuroticism,5603.5,4.410483e-02,6.615725e-01,False
